In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("data/train.csv", sep=";")
data

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder


In [5]:
emo_dict = {category : i for category, i in zip(data.Emotion.unique(), range(0, 6))}
data["EmoNum"] =  data.Emotion.apply(lambda x : int(emo_dict[x]))

emo_dict

{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}

In [36]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data[["Text"]], data["EmoNum"], test_size=0.25)


# encoder = OneHotEncoder()
# train_text = encoder.fit_transform(Xtrain).toarray()
# test_text = encoder.transform(Xtest).toarray()


vectorizer = CountVectorizer()
corpus = vectorizer.fit_transform(Xtrain.Text).toarray()
test_corpus = vectorizer.transform(Xtest.Text).toarray()

test_corpus

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [45]:
clf = MultinomialNB()
clf.fit(corpus, ytrain)

clf.score(test_corpus, ytest)

0.737

In [81]:
prompt = "im not "
pcorpus = vectorizer.transform([prompt]).toarray()

clf.predict(pcorpus)

array([0], dtype=int64)

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vector = TfidfVectorizer(analyzer="word", sublinear_tf=True)
corp2 = tfid_vector.fit_transform(Xtrain.Text).toarray()
tcorp2 = tfid_vector.transform(Xtest.Text).toarray()


In [65]:
clf2 = MultinomialNB()
clf2.fit(corp2, ytrain)

clf2.score(tcorp2, ytest)

0.63775

In [70]:
import pickle

with open("data/model_without_norm.pkl", "wb") as f:
    pickle.dump(clf, f)

    

In [72]:
from nltk.stem.wordnet import WordNetLemmatizer

lemma = WordNetLemmatizer()


def NormalizeText(tweet:str):
    mod_tweet = ""
    for word in tweet:
        if (not word.isalpha) : continue
        mod_tweet += lemma.lemmatize(word).lower()
        # mod_tweet += (st.stem(word)).lower()

    return mod_tweet


data2 = data.copy(deep=True)

In [78]:
data2.Text = data2.Text.apply(lambda x : NormalizeText(x))

In [79]:
# Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(data2[["Text"]], data2["EmoNum"], test_size=0.25)

vectorizer2 = CountVectorizer()
ncorpus = vectorizer.fit_transform(Xtrain.Text).toarray()
n_test_corpus = vectorizer.transform(Xtest.Text).toarray()

ncorpus

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [6]:
test_data = pd.read_csv("data/test.csv", sep=";")
test_data


test_data["EmoNum"] =  test_data.Emotion.apply(lambda x : int(emo_dict[x]))


vectorizer3 = CountVectorizer()

corpp = vectorizer3.fit_transform(data.Text).toarray()
test_corpp = vectorizer3.transform(test_data.Text).toarray()

corpp


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [9]:
ytrain = data.EmoNum
ytest = test_data.EmoNum

newclf = MultinomialNB()
newclf.fit(corpp, ytrain)

newclf.score(test_corpp, ytest)

0.7655

In [11]:
p = "hey im happy"

corpus_p = vectorizer3.transform([p]).toarray()
newclf.predict(corpus_p)

0